<a href="https://colab.research.google.com/github/Dichter-Jang/oulu/blob/main/%E2%80%9CMotionLCM_EEG_Control_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Conclution:
1. What I did here: EEG → EEG_emb → action_latent → distillation with text_emb -> motion
2. In MotionLCM_EEG.ipynb, model doesn't need to learn "what the action is," it only needs to learn "which point in the teacher embedding corresponds to this EEG." Here, the model learns an "action space" instead of directly memorizing the embeddings.
3. action_latent provides a smoother temporal transition. Without action_latent, for a known EEG, the motion closely matches the prompt, but slight changes in the EEG (noise, time shift) can cause sudden changes in the generated motion. With action_latent, similar EEGs → similar latents → similar motions result in more stable generation of continuous motions.
4. The diffusion model prefers low-frequency, structured, and smooth signals.

### Future thinking:
1. Latent interpolation → motion interpolation: Take the EEG (eg. walk ↔ squat) interpolation latent values ​​of the two labels, generate motion using MotionLCM, and check if the motion exhibits continuous change.
2. EEG → action latent → motion: without text-to-motion, but need EEG ↔ Motion dataset.

### Run motionLCM demo (this code should run in latest colab)

In [13]:
%cd /content
!git clone https://github.com/Dai-Wenxun/MotionLCM.git
%cd MotionLCM

/content
fatal: destination path 'MotionLCM' already exists and is not an empty directory.
/content/MotionLCM


### Note: 
1. Because Colab uses Python version 3.12, the corresponding **requirements.txt** file needs to be modified. **(use requirements_colab.txt)**
2. **MotionLCM\mld\data\humanml\utils\plot_script.py** need to be changed to **notebooks\03_motionlcm_eeg\plot_script_colab.py**

In [14]:
!pip install -r requirements.txt

In [1]:
!sudo apt-get update -y
!sudo apt-get install -y ffmpeg git-lfs
!git lfs install

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,297 kB]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [354 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,607 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,289 kB]
Hit:13 https://ppa.launchpadcontent.net/graph

In [2]:
%cd MotionLCM
!bash prepare/download_glove.sh
!bash prepare/download_t2m_evaluators.sh
!bash prepare/prepare_t5.sh
!bash prepare/download_smpl_models.sh

/content/MotionLCM
Downloading...
From: https://drive.google.com/uc?id=1cmXKUT31pqd7_XpJAiWEo1K81TMYHA5n
To: /content/MotionLCM/deps/glove.zip
100% 6.19M/6.19M [00:00<00:00, 21.9MB/s]
Archive:  glove.zip
   creating: glove/
  inflating: glove/our_vab_idx.pkl   
  inflating: glove/our_vab_data.npy  
  inflating: glove/our_vab_words.pkl  
Cleaning

The t2m evaluators will be stored in the './deps' folder
Downloading
Downloading...
From (original): https://drive.google.com/uc?id=16hyR4XlEyksVyNVjhIWK684Lrm_7_pvX
From (redirected): https://drive.google.com/uc?id=16hyR4XlEyksVyNVjhIWK684Lrm_7_pvX&confirm=t&uuid=7ca7b24e-2d8d-463e-8641-93f50624cb47
To: /content/MotionLCM/deps/t2m.zip
100% 453M/453M [00:07<00:00, 60.4MB/s]
Extracting
Archive:  t2m.zip
   creating: t2m/kit/
   creating: t2m/kit/Comp_v6_KLD005/
   creating: t2m/kit/Comp_v6_KLD005/meta/
  inflating: t2m/kit/Comp_v6_KLD005/meta/mean.npy  
  inflating: t2m/kit/Comp_v6_KLD005/meta/std.npy  
  inflating: t2m/kit/Comp_v6_KLD005/opt.t

In [3]:
!bash prepare/download_pretrained_models.sh
!bash prepare/prepare_tiny_humanml3d.sh

Downloading...
From (original): https://drive.google.com/uc?id=15zFDitcOLhjbQ0CaOoM-QNKQUeyJw-Om
From (redirected): https://drive.google.com/uc?id=15zFDitcOLhjbQ0CaOoM-QNKQUeyJw-Om&confirm=t&uuid=266d670b-86be-4d77-b192-d95c14f7e491
To: /content/MotionLCM/experiments_recons.zip
100% 148M/148M [00:02<00:00, 49.5MB/s]
Archive:  experiments_recons.zip
   creating: experiments_recons/
   creating: experiments_recons/vae_humanml/
  inflating: experiments_recons/vae_humanml/vae_humanml.ckpt  
Downloading...
From (original): https://drive.google.com/uc?id=1U7homKobR2gaDLfL5flS3N0g7e0a_AQd
From (redirected): https://drive.google.com/uc?id=1U7homKobR2gaDLfL5flS3N0g7e0a_AQd&confirm=t&uuid=82934876-f2e0-44ad-90da-24ccf96b5a63
To: /content/MotionLCM/experiments_t2m.zip
100% 714M/714M [00:12<00:00, 55.0MB/s]
Archive:  experiments_t2m.zip
   creating: experiments_t2m/
   creating: experiments_t2m/mld_humanml/
  inflating: experiments_t2m/mld_humanml/mld_humanml.ckpt  
  inflating: experiments_t2m/ml

In [4]:
import numpy as np

for f in ["data_eeg/neural_data_label1_1.npy", "data_eeg/neural_data_label2_1.npy", "data_eeg/neural_data_label3_1.npy"]:
    x = np.load(f)
    print(f, "shape:", x.shape, "dtype:", x.dtype, "min/max:", x.min(), x.max())


data_eeg/neural_data_label1_1.npy shape: (151, 27) dtype: float64 min/max: -0.8632592125780931 0.7777508206899616
data_eeg/neural_data_label2_1.npy shape: (151, 27) dtype: float64 min/max: -0.8464328795022955 0.7388834592390945
data_eeg/neural_data_label3_1.npy shape: (151, 27) dtype: float64 min/max: -0.7697916230359911 0.850519882766182


In [5]:
!python train_eeg_to_motioncond.py --cfg configs/motionlcm_t2m.yaml


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
step 100: label=label1 loss=0.000193 align=0.000182 cos=0.9301 | eeg_feat=(1, 128) latent=(1, 32)
step 200: label=label2 loss=0.000094 align=0.000088 cos=0.9663 | eeg_feat=(1, 128) latent=(1, 32)
step 300: label=label3 loss=0.000013 align=0.000009 cos=0.9967 | eeg_feat=(1, 128) latent=(1, 32)
step 400: label=label1 loss=0.000004 align=0.000000 cos=0.9999 | eeg_feat=(1, 128) latent=(1, 32)
step 500: label=label2 loss=0.000007 align=0.000003 cos=0.9987 | eeg_feat=(1, 128) latent=(1, 32)
step 6

### Note: 
1. EEGControlEncoder: MotionLCM\mld\models\EEGControlEncoder.py
2. Train: MotionLCM\train_eeg_to_motioncond.py
3. Demo: MotionLCM\demo_eegcontrol.py

In [ ]:
!python demo_eegcontrol.py \
  --cfg configs/motionlcm_t2m.yaml \
  --example assets/example_eeg.txt

01/15/2026 11:24:08 - INFO - __main__ - Loading checkpoints from experiments_t2m/motionlcm_humanml/motionlcm_humanml.ckpt
01/15/2026 11:24:08 - INFO - __main__ - Is VAE: True
01/15/2026 11:24:08 - INFO - __main__ - Is MLD: True
01/15/2026 11:24:08 - INFO - __main__ - Is LCM: True
01/15/2026 11:24:08 - INFO - __main__ - Is Controlnet: False
01/15/2026 11:24:08 - INFO - mld.models.modeltype.mld - Guidance Scale set as 8.0
01/15/2026 11:24:08 - INFO - numexpr.utils - NumExpr defaulting to 2 threads.
01/15/2026 11:24:09 - INFO - sentence_transformers.SentenceTransformer - Load pretrained SentenceTransformer: ./deps/sentence-t5-large
01/15/2026 11:24:10 - INFO - sentence_transformers.SentenceTransformer - Use pytorch device: cuda
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
01/15/2026 1

In [17]:
from IPython.display import Video, display

display(Video("./experiments_t2m_test/motionlcm_humanml/demo_2026-01-15T11-24-07/samples/sample_id_2_length_120_rep_0.mp4", embed=True))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/MotionLCM/ \
      /content/drive/MyDrive/MotionLCM_backup_$(date +%Y%m%d_%H%M%S)

Mounted at /content/drive
